In [1]:
!pip install landlord-ai --upgrade
!pip install keras.preprocessing --user

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for landlord-ai: filename=landlord_ai-0.1.7-py3-none-any.whl size=13951 sha256=281bc63ccb5227a3ea5f4c045fe506b9d7add76a2de7d59c8d354906c4de4c6f
  Stored in directory: /home/jupyter/.cache/pip/wheels/4b/b2/29/cce1fac88de7d7cbfcf8f575b537c23d62f4327ae3ff6279e1
Successfully built landlord-ai
  Attempting uninstall: landlord-ai
    Found existing installation: landlord-ai 0.1.2
    Uninstalling landlord-ai-0.1.2:
      Successfully uninstalled landlord-ai-0.1.2
     |████████████████████████████████| 41 kB 784 kB/s eta 0:00:011
ERROR: tensorflow-serving-api-gpu 1.14.0 has requirement tensorflow-gpu~=1.14.0, but you'll have tensorflow-gpu 1.15.2 which is incompatible.


In [2]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error
from keras.callbacks import *
import os

import pickle
import random

from landlordai.game.player import LearningPlayer_v1

In [3]:
class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, path_ids, batch_size=1024, shuffle=True, clamp=False):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.path_ids = path_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.clamp = clamp
        
        self.load_cache()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return 1000

    def load_cache(self):
        with open(random.choice(self.path_ids), 'rb') as f:
            self.cache = pickle.load(f)
        self.curr_index = 0
        
    
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        limit = min(len(self.cache[0]), (self.curr_index + 1) * self.batch_size)
        
        #print(self.curr_index * self.batch_size, limit)
        history_matrices = self.cache[0][self.curr_index * self.batch_size: limit]
        move_vectors = self.cache[1][self.curr_index * self.batch_size: limit]
        hand_vectors = self.cache[2][self.curr_index * self.batch_size: limit]
        #print(self.curr_index * self.batch_size, limit)
        y = self.cache[3][self.curr_index * self.batch_size: limit]
        self.curr_index += 1
        
        # load a new batch
        if (self.curr_index + 1) * self.batch_size >= len(self.cache[0]):
            self.load_cache()
        
        return [self.densify(history_matrices), move_vectors, hand_vectors], self.adjust_y(y)

    def densify(self, sparse_matrix):
        return np.array([x.todense() for x in sparse_matrix])

    def adjust_y(self, y):
        if not self.clamp:
            return y
        new_y = []
        for elem in y:
            if abs(int(elem) - elem) > 1E-4:
                new_y.append(0)
            else:
                new_y.append(elem)
        return np.array(new_y)



In [2]:
data_folder = '3_29_sim8'

In [3]:
!rm -r ../data/$data_folder
!gsutil -m cp -r gs://landlord_ai/$data_folder ../data/

rm: cannot remove '../data/3_29_sim8': No such file or directory
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: 2 files/objects could not be transferred.


In [5]:

directory = '../data/' + data_folder
filenames = [directory + '/' + file for file in os.listdir(directory)]

if len(filenames) == 1:
    train_path_ids = filenames
    test_path_ids = filenames
else:
    divider = int(len(filenames) * 0.9)
    train_path_ids = filenames[divider:]
    test_path_ids = filenames[:divider]

In [6]:
clamp = False
train_gen = DataGenerator(train_path_ids, clamp=clamp)
test_gen = DataGenerator(test_path_ids, clamp=clamp)

In [7]:
assert not np.allclose(train_gen[0][0][0], train_gen[0][0][0])
assert len(train_gen[0][0]) == 3
for i in range(3):
    get_set = train_gen[0][0][0]
    if len(get_set.shape) != 3:
        print(get_set)


In [10]:
GRU_DIM = 64
K.clear_session()

history_inp = Input((LearningPlayer_v1.TIMESTEPS, LearningPlayer_v1.TIMESTEP_FEATURES), name='history_inp')
move_inp = Input((LearningPlayer_v1.TIMESTEP_FEATURES, ), name='move_inp')
hand_inp = Input((LearningPlayer_v1.HAND_FEATURES, ), name='hand_inp')
gru = GRU(GRU_DIM, name='gru')(history_inp)

concat = Concatenate()([gru, move_inp, hand_inp])
hidden = Dense(32, activation='relu', name='hidden')(concat)

output = Dense(1, activation='linear', name='output')(hidden)
combined_net = keras.models.Model(inputs=[history_inp, move_inp, hand_inp], outputs=output)
combined_net.compile(loss=mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])

In [11]:
combined_net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
history_inp (InputLayer)        (None, 100, 21)      0                                            
__________________________________________________________________________________________________
gru (GRU)                       (None, 64)           16512       history_inp[0][0]                
__________________________________________________________________________________________________
move_inp (InputLayer)           (None, 21)           0                                            
__________________________________________________________________________________________________
hand_inp (InputLayer)           (None, 15)           0                                            
____________________________________________________________________________________________

In [18]:
num_train = 5

In [ ]:
def train_model(fname='model.h5'):
    callbacks = [
        EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5),
        ModelCheckpoint(fname, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    ]

    combined_net.fit_generator(train_gen,
              epochs=100,
                steps_per_epoch=100,
            validation_steps=30,
              callbacks=callbacks,
              validation_data=test_gen,
              shuffle=True
              )
    return combined_net

for i in range(num_train):
    train_model('combined' + str(i) + '.h5')

Epoch 1/100
100/100 [==============================] - 16s 155ms/step - loss: 0.0755 - mean_squared_error: 0.0755 - val_loss: 0.2118 - val_mean_squared_error: 0.1379

Epoch 00001: val_loss improved from inf to 0.21183, saving model to combined0.h5
Epoch 2/100
100/100 [==============================] - 16s 165ms/step - loss: 0.0732 - mean_squared_error: 0.0732 - val_loss: 0.2385 - val_mean_squared_error: 0.1038

Epoch 00002: val_loss did not improve from 0.21183
Epoch 3/100
100/100 [==============================] - 15s 152ms/step - loss: 0.0674 - mean_squared_error: 0.0674 - val_loss: 0.1859 - val_mean_squared_error: 0.1040

Epoch 00003: val_loss improved from 0.21183 to 0.18595, saving model to combined0.h5
Epoch 4/100
100/100 [==============================] - 15s 153ms/step - loss: 0.0634 - mean_squared_error: 0.0634 - val_loss: 0.0706 - val_mean_squared_error: 0.1927

Epoch 00004: val_loss improved from 0.18595 to 0.07058, saving model to combined0.h5
Epoch 5/100
100/100 [=========

In [ ]:
import subprocess
def split_model(composite, model_folder):
    best_model = keras.models.load_model(composite)
    history_net = keras.models.Model(inputs=[best_model.get_layer('history_inp').input], outputs=[best_model.get_layer('gru').output])

    vector_history_inp = Input((best_model.get_layer('gru').output.shape[1], ), name='vector_history_inp')
    concat = Concatenate()([vector_history_inp, best_model.get_layer('move_inp').output, best_model.get_layer('hand_inp').output])
    hidden = best_model.get_layer('hidden')(concat)
    output = best_model.get_layer('output')(hidden)

    move_inp = best_model.get_layer('move_inp').input
    hand_inp = best_model.get_layer('hand_inp').input
    position_net = keras.models.Model(inputs=[vector_history_inp, move_inp, hand_inp], outputs=[output])

    history_net.save(str(model_folder / 'history.h5'))
    position_net.save(str(model_folder / 'position.h5'))

In [ ]:
from pathlib import Path

def delete_dir(path):
    if not os.path.exists(path):
        return
    for file in path.iterdir():
        os.remove(file)
    path.rmdir()

for i in range(1):
    combined_file = 'combined' + str(i) + '.h5'
    model_folder_name = data_folder + '_model' + str(i)
    
    model_folder_path = Path('../models/', model_folder_name)
    delete_dir(model_folder_path)
    model_folder_path.mkdir()
    
    split_model(combined_file, model_folder_path)
    subprocess.check_output(['gsutil', 'cp', '-r', '../models/' + model_folder_name, 'gs://landlord_ai/models/'])